In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
df_anemia = pd.read_csv("/content/TB_DIGTEL_ANEMIA_ATENDIDOS.csv")
df_anemia.head()

In [3]:
df_anemia.dropna(inplace=True)

In [ ]:
df_anemia = (
    df_anemia.assign(
        sexo_num = lambda df: df.Sexo.replace(["F","M"],[0,1])
    )
)
df_anemia

In [ ]:
df_anemia["anemia"] = df_anemia["Diagnostico"].apply(lambda x: 1 if x=="D509" else 0)
df_anemia

In [ ]:
# TODOs
# 1. Correlaciones (heatmap)
# 2. Propuesta de modelo
# 3. Matríz de Confusión
# 4. Exactitud